# Instagram Sentiment Analysis POC - Transformer trained

In [1]:
import pandas as pd
import re
import nltk
from langdetect import detect, DetectorFactory
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline
from sklearn.model_selection import train_test_split
import torch

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shyam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shyam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_csv("Instagram-datasets.csv")

df = df.dropna(subset=['comment']).reset_index(drop=True)

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df["is_english"] = df["comment"].apply(is_english)
df = df[df["is_english"]]

# Clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|@\w+|#\w+|[^\x00-\x7F]+", " ", text)  # Removing url using regex on HTTP, @ regex mentions, and emoji code regex patterns
    text = re.sub(r"[^a-z\s]", "", text)                          # Letter Only Regex
    text = re.sub(r"\s+", " ", text).strip()                     # Space Trim Logic
    return text

df["clean_comment"] = df["comment"].apply(clean_text)
df["label"] = -1  

In [ ]:
df_labeled = df.sample(frac=1.0, random_state=42).copy()
mid = len(df_labeled) // 2
df_labeled.iloc[:mid, df_labeled.columns.get_loc("label")] = 0  
df_labeled.iloc[mid:, df_labeled.columns.get_loc("label")] = 1  

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_labeled["clean_comment"].tolist(),
    df_labeled["label"].tolist(),
    test_size=0.2,
    random_state=42
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [6]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [7]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    logging_dir="./logs",
    save_steps=500,
    eval_steps=500,
    save_total_limit=2
)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\shyam\Downloads\Project_NLP_Insta\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=54, training_loss=0.6918144932499638, metrics={'train_runtime': 131.0724, 'train_samples_per_second': 6.363, 'train_steps_per_second': 0.412, 'total_flos': 39429658311840.0, 'train_loss': 0.6918144932499638, 'epoch': 3.0})

In [11]:
model_path = "bert-instagram-sentiment"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('bert-instagram-sentiment\\tokenizer_config.json',
 'bert-instagram-sentiment\\special_tokens_map.json',
 'bert-instagram-sentiment\\vocab.txt',
 'bert-instagram-sentiment\\added_tokens.json',
 'bert-instagram-sentiment\\tokenizer.json')

In [12]:
sentiment_pipe = TextClassificationPipeline(
    model=AutoModelForSequenceClassification.from_pretrained(model_path),
    tokenizer=AutoTokenizer.from_pretrained(model_path),
    return_all_scores=False,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cpu
c:\Users\shyam\Downloads\Project_NLP_Insta\.venv\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [13]:
print(sentiment_pipe("I love this post."))
print(sentiment_pipe("This is a horrible idea!"))

[{'label': 'LABEL_1', 'score': 0.5331133008003235}]
[{'label': 'LABEL_1', 'score': 0.5153830647468567}]
